In [53]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Extract Data from time period 

In [32]:
raw_data = pd.read_csv('hotel_bookings.csv')

In [52]:
def clean_data(raw_data):
    #--> Map the name of the months with their corresponding numbers
    def map_months(x):
        if x == 'January':
            x = 1
        elif x == 'February':
            x = 2
        elif x == 'March':
            x = 3
        elif x == 'April':
            x = 4
        elif x == 'May':
            x = 5
        elif x == 'June':
            x = 6
        elif x == 'July':
            x = 7
        elif x == 'August':
            x = 8
        elif x == 'September':
            x = 9
        elif x == 'October':
            x = 10
        elif x == 'November':
            x = 11
        elif x == 'December':
            x = 12
        return x

    # Use the above mapping to create a new feature
    raw_data['arrival_date_month_number'] = raw_data['arrival_date_month'].apply(lambda x : map_months(x))
    raw_data['arrival_date'] = raw_data['arrival_date_day_of_month'].astype('str') + '/' + raw_data['arrival_date_month_number'].astype('str') + '/'+ raw_data['arrival_date_year'].astype('str')
    # Convert into datetime variable
    raw_data['arrival_date'] = pd.to_datetime(raw_data['arrival_date'])
    #compute number of days each stay is 
    raw_data['length_stay_days'] = raw_data['stays_in_week_nights'] + raw_data['stays_in_weekend_nights']
    #compute number occupants
    raw_data['occupants'] = raw_data['adults'] + raw_data['children']
    #get rid of samples with length_stay == 0 or adr == 0 
    raw_data = raw_data[raw_data['length_stay_days']>=1]
    raw_data = raw_data[raw_data['adr'] > 0]
    #get rid of a few samples where country is na 
    raw_data = raw_data[~raw_data['country'].isna()]

    return raw_data

In [34]:
cleaned_data = clean_data(raw_data)

In [76]:
cleaned_data.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date',
       'arrival_date_month_number', 'arrival_date', 'length_stay_days',
       'occupants'],
      dtype='object')

In [ ]:
"""
data cleaning notes: 

-remove bookings with length_stay_days == 0  (only 715 samples)
-remove bookings with adr == (1959 samples)
-remove bookings with na in the following: country (only 488 samples)
"""

In [51]:
def generate_prescriptive_params(raw_data,start_day,time_window,selected_features):
    """
    Function to extract all the parameters we need for the prescriptive model, which will be run in Julia.
    
    @param raw_data: dataframe with raw bookings data. Must have the column 'arrival_date' which is a datetime variable
    @param start_day: first day of window we consider for accepting/rejecting bookings.
                      format should be string of the form "<year>-<month>-<day>"
    @param time_window: Number of days we add onto start_day to create the time window we consider. We get data on 
                        every booking with an arrival date which falls into our window.
    @param selected_features: list of features to pull for each booking. Should match whatever predictive model
                              we trained 
                    
    @return p: vector length n. The (total) price of each booking 
    @return s: vector length n (integers in {0,...,T }). The starting day of each booking
    @return e: vector length n (integers in {0,...,T}). The ending day of each booking
    @return t: vector length n. This gives the number of people that each booking is for
    @return X: n x p matrix containing selected features of the booking
    @return y: whether the booking was canceled or not
    """    
    start_day = datetime.strptime(start_day,'%Y-%m-%d')
    #get bookings in specified time window
    window_data = raw_data[(raw_data['arrival_date'] >=  start_day) &
                           (raw_data['arrival_date'] <= start_day + timedelta(days=time_window))]
    window_data = window_data.sort_values(by='arrival_date',ascending=True)
    print("window_data shape", window_data.shape)
    #compute arrival date as an int ()
    timedeltas_s = window_data['arrival_date'] - start_day
    print("time_deltas_s len: ", len(timedeltas_s))
    s = pd.Series([int(x.days) for x in timedeltas_s])
    print("s shape: ", s.shape)
    #compute end date as an int
    print("len window_data[length_stay_days] shape",(window_data['length_stay_days']).shape)
    e = np.array(s) + np.array(window_data['length_stay_days'])
    print("e len: ", len(e))
    p = window_data['adr']*window_data['length_stay_days']
    t = window_data['occupants']
    X = pd.get_dummies(window_data[selected_features])
    y = window_data['is_canceled']
    return (window_data, np.array(s), np.array(e), 
           np.array(p), np.array(t), X, y)
    

# Fit kNN model

In [54]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, normalize

In [48]:
def get_training_data(raw_data,start_date,end_date,selected_features):
    """
    Function to generate training data for a predictive model (OCT,kNN)
    which will later be used for prescriptions
    
    @param raw_data: dataframa holding raw booking data
    @param start_date: format "<year>-<month>-<day>"
    @param end_date: format "<year>-<month>-<day>"
    @param selected_features: list of features model should be fitted on 
    """
    start_date = datetime.strptime(start_date,'%Y-%m-%d')
    end_date = datetime.strptime(end_date,'%Y-%m-%d')
    #get bookings in specified time window
    window_data = raw_data[(raw_data['arrival_date'] >=  start_date) &
                           (raw_data['arrival_date'] <= end_date)]
    window_data = window_data.sort_values(by='arrival_date',ascending=True)
    X = window_data[selected_features]
    X = pd.get_dummies(X)
    y = np.array(window_data['is_canceled'])
    return X,y

In [81]:
def get_prescriptive_params_knn(data,start_date_knn,end_date_knn,
                                start_date_test,time_window,
                                selected_features):
    window_data, s, e, p, t, X_new, y_new = generate_prescriptive_params(data,start_date_test,time_window,selected_features)

    X,y = get_training_data(raw_data,start_date_knn,end_date_knn,selected_features)
    nbrs = NearestNeighbors(n_neighbors=5).fit(X)
    distance, indices = nbrs.kneighbors(X_new)
    y_neighbors = y[indices]
    
    return window_data, s, e, p, t, X_new, np.array(y_new), y_neighbors


In [85]:
start_date_knn = "2017-01-01"
end_date_knn = "2017-07-30"
start_date_test = "2017-08-01"
time_window = 21
selected_features = ['lead_time','length_stay_days']

window_data, s, e, p, t, X_new, y_new, y_neighbors =  get_prescriptive_params_knn(cleaned_data,start_date_knn,end_date_knn,
                                                                                    start_date_test,time_window,
                                                                                    selected_features)

window_data shape (2930, 36)
time_deltas_s len:  2930
s shape:  (2930,)
len window_data[length_stay_days] shape (2930,)
e len:  2930


In [86]:
np.savetxt('e.csv',e,delimiter=',')
np.savetxt('s.csv',s,delimiter=',')
np.savetxt('p.csv',p,delimiter=',')
np.savetxt('t.csv',t,delimiter=',')
X_new.to_csv('X_new.csv',header=True)
np.savetxt('y_new.csv',y_new,delimiter=',')
np.savetxt('y_neighbors.csv',y_neighbors,delimiter=',')